## Difference Project

Some useful utility function.

In [1]:
function SoftThreshold(z, lambda)
    abs(z) < lambda ? 0. : z > 0 ? z - lambda : z + lambda
end

@show SoftThreshold(3, 1)
@show SoftThreshold(-3, 1)
@show SoftThreshold(-0.5, 1);

SoftThreshold(3,1) => 2
SoftThreshold(-3,1) => -2
SoftThreshold(-0.5,1) => 0.0


Generate data and compute precision matrices

In [100]:
p = 200
Sigmax = eye(p,p)
Sigmay = zeros(p,p)
rho = 0.7
for i=1:p
    for j=1:p
        Sigmay[i,j]=rho^abs(i-j)
    end
end

sqmy = sqrtm(Sigmay)

n = 1000
X = randn(n,p)
Y = randn(n,p) * sqmy;

hSx = cov(X, corrected=false)
hSy = cov(Y, corrected=false);

In [101]:
B = inv(Sigmax) - inv(Sigmay)
B[1:5,1:5]

5x5 Array{Float64,2}:
 -0.960784      1.37255      -8.59879e-17  -1.6218e-16    2.76467e-16
  1.37255      -1.92157       1.37255       2.28575e-16  -6.00827e-16
 -1.11022e-16   1.37255      -1.92157       1.37255       4.76743e-16
 -1.88738e-16   1.11022e-16   1.37255      -1.92157       1.37255    
  4.996e-16    -6.32827e-16   1.11022e-16   1.37255      -1.92157    

Solve the problem using Convex and Mosek for comparison purposes

In [6]:
using Convex
using Mosek

solver = MosekSolver(LOG=1)
set_default_solver(solver);

In [7]:
lambda = 0.2

Delta = Variable(p,p);
prob = minimize(quad_form(vec(Delta), kron(hSy,hSx)) / 2 - trace((hSy-hSx)*Delta) +  lambda * norm(vec(Delta), 1))
prob.constraints += [Delta == Delta']
@time solve!(prob)

sol = Delta.value

Computer
  Platform               : Linux/64-X86    
  Cores                  : 2               

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3606            
  Cones                  : 2               
  Scalar variables       : 2707            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Conic interior-point optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Total number of eliminations : 901
Eliminator terminated.
Eliminator started.
Total number of eliminations : 901
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Eliminator - elim's                 : 901             
Lin. dep.  - tries                  : 1                 time    

30x30 Array{Float64,2}:
 -0.0905859     0.363101    …   2.46919e-8  2.54151e-8  1.70861e-8
  0.363101     -1.34289e-7      1.86669e-8  1.55938e-8  6.88327e-9
  1.35761e-8    2.32714e-8      1.16153e-8  1.23632e-8  3.91681e-8
  2.03284e-8    3.07495e-8      2.50669e-8  1.68967e-8  1.22578e-8
  2.93835e-8    1.92833e-8      1.99163e-8  2.6433e-9   1.8519e-8 
  1.37238e-8    2.9293e-8   …   3.6501e-8   3.55841e-8  8.4374e-9 
  1.04088e-8    1.53266e-8      1.82709e-8  1.43923e-8  1.799e-8  
  1.42638e-8    1.86234e-8     -1.02943e-8  1.98548e-8  1.654e-8  
  2.4799e-8     3.95952e-8      1.12199e-8  2.30331e-8  2.19327e-8
  1.50942e-8    1.9854e-8       1.37287e-8  1.51906e-8  2.3015e-8 
  2.5422e-8     1.64783e-8  …   1.91705e-8  2.17802e-8  5.64587e-8
  1.81276e-8    2.22423e-8      2.59481e-8  2.60715e-8  2.97696e-8
  4.50347e-10   2.45227e-8      5.12902e-9  2.32278e-8  2.438e-8  
  ⋮                         ⋱                                     
  2.43916e-8    2.5174e-8       2.1029

Shooting algorithm for the objective

In [61]:
function DiffCovShooting(hSx, hSy, lambda, Ups; maxIter=2000, optTol=1e-7)
    p = size(hSx, 1)
        
    Delta = zeros(p, p)
    A = zeros(p, p)
    
    iter = 1;
    while iter < maxIter    
        if mod(iter, 10) == 0
            @show "outer $iter"
        end
    
        fDone = true
        for a=1:p
            for b=a:p
                if a==b
                    # diagonal elements
                    x0 = hSx[a,a]*hSy[a,a]/2
                    x1 = A[a,a] - hSy[a,a] + hSx[a,a]
                    tmp = SoftThreshold(-x1/x0/2 + Delta[a,b], lambda*Ups[a,b] / 2 / x0)
                else
                    # off-diagonal elements
                    x0 = (hSx[a,a]*hSy[b,b] + hSx[b,b]*hSy[a,a])/2 + hSx[a,b]*hSy[a,b]
                    x1 = A[a,b] + A[b,a] - 2*(hSy[a,b] - hSx[a,b])
                    tmp = SoftThreshold(-x1/x0/2 + Delta[a,b], lambda*Ups[a,b] / x0)
                end
                
                h = tmp - Delta[a,b]
                Delta[a,b] = tmp
                Delta[b,a] = tmp
                if abs(h) > optTol
                    fDone = false
                end
                for j=1:p
                    for k=1:p
                        if a == b
                            A[j,k] = A[j,k] + h * hSx[j,a]*hSy[a,k]
                        else                            
                            A[j,k] = A[j,k] + h * (hSx[j,a]*hSy[b,k] + hSx[j,b]*hSy[a,k])
                        end
                    end
                end
            end
        end
        
        iter = iter + 1;
        if fDone
            break
        end
    end
    Delta
end

DiffCovShooting (generic function with 1 method)

In [102]:
lambda = 0.2
@time solShoot = DiffCovShooting(hSx, hSy, lambda, ones(p,p))

"outer $(iter)" => "outer 10"
"outer $(iter)" => "outer 20"
"outer $(iter)" => "outer 30"
elapsed time: 249.31510672 seconds (1003120 bytes allocated)


200x200 Array{Float64,2}:
 -0.16258    0.582703   0.0       …   0.0          0.0        0.0       
  0.582703  -0.63669    0.72368       0.0          0.0        0.0       
  0.0        0.72368   -0.971444      0.0          0.0        0.0       
  0.0        0.0        0.763628      0.0          0.0        0.0       
  0.0        0.0        0.0           0.0          0.0        0.0       
  0.0        0.0        0.0       …   0.0          0.0        0.0       
  0.0        0.0        0.0           0.0          0.0        0.0       
  0.0        0.0        0.0           0.0          0.0        0.0       
  0.0        0.0        0.0           0.0          0.0        0.0       
  0.0        0.0        0.0           0.0          0.0        0.0       
  0.0        0.0        0.0       …   0.0          0.0        0.0       
  0.0        0.0        0.0           0.0          0.0        0.0       
  0.0        0.0        0.0           0.0          0.0        0.0       
  ⋮                      

In [92]:
solShoot[1:7,1:7]

7x7 Array{Float64,2}:
 0.0        0.363742   0.0        0.0        0.0        0.0  0.0
 0.363742  -0.235062   0.323915   0.0        0.0        0.0  0.0
 0.0        0.323915  -0.16835    0.353804   0.0        0.0  0.0
 0.0        0.0        0.353804  -0.459889   0.474936   0.0  0.0
 0.0        0.0        0.0        0.474936  -0.0387662  0.0  0.0
 0.0        0.0        0.0        0.0        0.0        0.0  0.0
 0.0        0.0        0.0        0.0        0.0        0.0  0.0

In [11]:
@show dot(vec(sol), kron(hSy,hSx)*vec(sol)) / 2 - trace((hSy-hSx)*sol) +  lambda * norm(vec(sol), 1)
@show dot(vec(solShoot), kron(hSy,hSx)*vec(solShoot)) / 2 - trace((hSy-hSx)*solShoot) +  lambda * norm(vec(solShoot), 1);

(dot(vec(sol),kron(hSy,hSx) * vec(sol)) / 2 - trace((hSy - hSx) * sol)) + lambda * norm(vec(sol),1) => -0.11710607022184877
(dot(vec(solShoot),kron(hSy,hSx) * vec(solShoot)) / 2 - trace((hSy - hSx) * solShoot)) + lambda * norm(vec(solShoot),1) => -0.11710919847566284


Implementation of the active shooting algorithm

In [109]:
function updateDelta!(Delta, hSx, hSy, A, lambda; maxIter=2000, optTol=1e-7)    
    # Delta is a sparse matrix stored in CSC format    
    # only diagonal and lower triangular elements are used
    
    # extract fields from sparse matrix Delta 
    colptr = Delta.colptr
    nzval = Delta.nzval
    rowval = Delta.rowval
    
    p = size(Delta, 1)
    
    iter = 1
    while iter <= maxIter
       if mod(iter, 100) == 0
            @show "inner $iter"
        end
        
        fDone = true
        for colInd=1:p
            for j=colptr[colInd]:colptr[colInd+1]-1
                rowInd = rowval[j]
                
                # a sanity check in 
                if rowInd < colInd
                    continue
                end
                
                if rowInd==colInd
                    # diagonal elements
                    x0 = hSx[rowInd,rowInd]*hSy[rowInd,rowInd]/2
                    x1 = A[rowInd,rowInd] - hSy[rowInd,rowInd] + hSx[rowInd,rowInd]
                    tmp = SoftThreshold(-x1/x0/2 + nzval[j], lambda[rowInd,colInd] / 2 / x0)
                else
                    # off-diagonal elements
                    x0 = (hSx[rowInd,rowInd]*hSy[colInd,colInd] + hSx[colInd,colInd]*hSy[rowInd,rowInd])/2 
                              + hSx[rowInd,colInd]*hSy[rowInd,colInd]
                    x1 = A[rowInd,colInd] + A[colInd,rowInd] - 2*(hSy[rowInd,colInd] - hSx[rowInd,colInd])
                    tmp = SoftThreshold(-x1/x0/2 + nzval[j], lambda[rowInd,colInd] / x0)
                end
                
                # size of update
                h = tmp - nzval[j]
                # update is too big -- not done
                if abs(h) > optTol
                    fDone = false
                end                
                # update Delta
                nzval[j] = tmp   
                
                # update A -- only active set
                for ci=1:p
                    for k=colptr[ci]:colptr[ci+1]-1
                        ri = rowval[k]
                        
                        if rowInd == colInd                                     
                            A[ri,ci] = A[ri,ci] + h * hSx[ri,rowInd]*hSy[rowInd,ci]
                            if ri != ci
                                A[ci,ri] = A[ci,ri] + h * hSx[ci,rowInd]*hSy[rowInd,ri]
                            end
                        else
                            A[ri,ci] = A[ri,ci] + h * (hSx[ri,colInd]*hSy[rowInd,ci] + hSx[ri,rowInd]*hSy[colInd,ci])
                            if ri != ci
                                A[ci,ri] = A[ci,ri] + h * (hSx[ci,colInd]*hSy[rowInd,ri] + hSx[ci,rowInd]*hSy[colInd,ri])
                            end                            
                        end
                    end
                end   # end update A                
            end
        end            
        
        iter = iter + 1;
        if fDone
            break
        end
        
    end  # while
    
    sparse(Delta)
end

function findViolator!(active_set, Delta, A, hSx, hSy, lambda; kktTol=1e-7)   
    p = size(hSx, 1)
    
    tmp = abs((A + A') / 2 - (hSy - hSx)) - lambda
    ind = indmax(tmp) 
    if tmp[ind] > kktTol
        push!(active_set, ind)
        Delta[ind] = eps()
    else
        ind = 0
    end
    
    return ind    
end

function DiffCovActiveShooting(hSx, hSy, lambda; maxIter=1000, maxInnerIter=1000, optTol=1e-7, Delta = [])
    p = size(hSx, 1)
        
    if isempty(Delta)
        Delta = spzeros(p, p)
        A = zeros(p, p)
        
        active_set = Array(Integer, 0)        
        indAdd = findViolator!(active_set, Delta, A, hSx, hSy, lambda)    
        if indAdd == 0
            return Delta
        end
    else
        lDelta = tril(Delta, -1)
        dDelta = spdiagm(diag(Delta))
        A = hSx * (lDelta + lDelta' + dDelta) * hSy
        
        # make sure Delta is lower triangular
        Delta = lDelta + dDelta
        active_set = find(Delta)
    end     
    
    
    
    iter = 1;
    while iter < maxIter    
        if mod(iter, 100) == 0
            @show "outer $iter"
        end
    
        old_active_set = copy(active_set)
        updateDelta!(Delta, hSx, hSy, A, lambda; maxIter=maxInnerIter, optTol=optTol)
        active_set = find(Delta)
        
        # update A
        lDelta = tril(Delta, -1)
        dDelta = spdiagm(diag(Delta))
        A = hSx * (lDelta + lDelta' + dDelta) * hSy
        # add violating element into active set 
        indAdd = findViolator!(active_set, Delta, A, hSx, hSy, lambda)         
        
        if old_active_set == active_set
            break
        else
        end       
        
        iter = iter + 1;
        
    end
    Delta
end

DiffCovActiveShooting (generic function with 1 method)

In [107]:
@time solShootFast = DiffCovActiveShooting(hSx, hSy, lambda*ones(p,p), optTol=1e-7)

"outer $(iter)" => "outer 10"
"outer $(iter)" => "outer 20"
"outer $(iter)" => "outer 30"
"outer $(iter)" => "outer 40"
"outer $(iter)" => "outer 50"
"outer $(iter)" => "outer 60"
"outer $(iter)" => "outer 70"
"outer $(iter)" => "outer 80"
"outer $(iter)" => "outer 90"
"outer $(iter)" => "outer 100"
"outer $(iter)" => "outer 110"
"outer $(iter)" => "outer 120"
"outer $(iter)" => "outer 130"
"outer $(iter)" => "outer 140"
"outer $(iter)" => "outer 150"
"outer $(iter)" => "outer 160"
"outer $(iter)" => "outer 170"
"outer $(iter)" => "outer 180"
"outer $(iter)" => "outer 190"
"outer $(iter)" => "outer 200"
"outer $(iter)" => "outer 210"
"outer $(iter)" => "outer 220"
"outer $(iter)" => "outer 230"
"outer $(iter)" => "outer 240"
"outer $(iter)" => "outer 250"
"outer $(iter)" => "outer 260"
"outer $(iter)" => "outer 270"
"outer $(iter)" => "outer 280"
"outer $(iter)" => "outer 290"
"outer $(iter)" => "outer 300"
"outer $(iter)" => "outer 310"
"outer $(iter)" => "outer 320"
"outer $(iter)" =

200x200 sparse matrix with 429 Float64 entries:
	[1  ,   1]  =  -0.16258
	[2  ,   1]  =  0.582703
	[2  ,   2]  =  -0.63669
	[3  ,   2]  =  0.72368
	[3  ,   3]  =  -0.971444
	[4  ,   3]  =  0.763628
	[113,   3]  =  0.0169449
	[4  ,   4]  =  -1.01247
	[5  ,   4]  =  0.840508
	[5  ,   5]  =  -0.809753
	⋮
	[196, 195]  =  0.717078
	[196, 196]  =  -0.827476
	[197, 196]  =  0.531696
	[197, 197]  =  -0.70556
	[198, 197]  =  0.80239
	[198, 198]  =  -0.905627
	[199, 198]  =  0.763004
	[200, 198]  =  0.00506813
	[199, 199]  =  -0.759454
	[200, 199]  =  0.545131
	[200, 200]  =  -0.193631

In [108]:
maximum(abs(tril(solShoot-solShootFast)))

1.76200985690933e-7

In [98]:
solShoot[1:7,1:7]

7x7 Array{Float64,2}:
 0.0        0.363742   0.0        0.0        0.0        0.0  0.0
 0.363742  -0.235062   0.323915   0.0        0.0        0.0  0.0
 0.0        0.323915  -0.16835    0.353804   0.0        0.0  0.0
 0.0        0.0        0.353804  -0.459889   0.474936   0.0  0.0
 0.0        0.0        0.0        0.474936  -0.0387662  0.0  0.0
 0.0        0.0        0.0        0.0        0.0        0.0  0.0
 0.0        0.0        0.0        0.0        0.0        0.0  0.0